In [125]:
import db.knowhere_db as kdb
import pipeline.pipeline as pipeline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split


In [126]:
glen_H_data_raw = pd.read_csv("Labeled_glen_2017-03-19 to 2017-03-23.csv")

In [127]:
test_labeled = pd.read_csv("glen_3_24_labeled.csv", index_col = 0)

In [128]:
glen_H_data = glen_H_data_raw[['Acceleration x','Acceleration y','Acceleration z', 'Altimeter (Barometer) Pressure',\
                               'Microphone Left Channel Level', 'Microphone Right Channel Level','Magnetometer x',\
                               'Magnetometer y','Magnetometer z','Gyrometer x','Gyrometer y',\
                               'Gyrometer z', 'classification']]
glen_H_data = glen_H_data.dropna()
#glen_H_data.head()

In [129]:
test = test_labeled[['Acceleration x','Acceleration y','Acceleration z', 'Altimeter (Barometer) Pressure',\
                               'Microphone Left Channel Level', 'Microphone Right Channel Level','Magnetometer x',\
                               'Magnetometer y','Magnetometer z','Gyrometer x','Gyrometer y',\
                               'Gyrometer z', 'classification']]
#test.head()

In [130]:
glen_H_data = glen_H_data.replace(to_replace='T_E', value='T_S')
glen_H_data = glen_H_data.replace(to_replace='E_U', value='E')
glen_H_data = glen_H_data.replace(to_replace='E_D', value='E')
glen_H_data = glen_H_data.replace(to_replace='S_U', value='W')
glen_H_data = glen_H_data.replace(to_replace='T_T', value='T_S')


glen_H_data.groupby("classification").size()

classification
D      1749
E       885
S       251
T_D    1159
T_S     789
W      1359
dtype: int64

In [131]:
test = test.replace(to_replace='T_E', value='T_S')
test = test.replace(to_replace='E_U', value='E')
test = test.replace(to_replace='E_D', value='E')
test = test.replace(to_replace='S_U', value='W')
test = test.replace(to_replace='T_T', value='T_S')


test.groupby("classification").size()

classification
D      150
E        8
S       37
T_D    297
T_S     84
W      255
dtype: int64

In [133]:
glen_H_data['Acceleration'] =  np.sqrt(glen_H_data['Acceleration x']**2 + glen_H_data['Acceleration y']**2 +\
                                       glen_H_data['Acceleration z']**2)
glen_H_data['Magnetometer'] =  np.sqrt(glen_H_data['Magnetometer x']**2 + glen_H_data['Magnetometer y']**2 +\
                                       glen_H_data['Magnetometer z']**2)
glen_H_data['Gyrometer']    =  np.sqrt(glen_H_data['Gyrometer x']**2 + glen_H_data['Gyrometer y']**2 +\
                                       glen_H_data['Gyrometer z']**2)
glen_H_data['Microphone']    =  (glen_H_data['Microphone Left Channel Level'] +\
                                glen_H_data['Microphone Right Channel Level'])/2

glen_H_data_labels = glen_H_data['classification']

glen_H_data = glen_H_data[['Acceleration','Magnetometer'
                           #'Gyrometer','Microphone','Altimeter (Barometer) Pressure', 
                           ]]

### Prep Test Data

In [134]:
test['Acceleration'] =  np.sqrt(test['Acceleration x']**2 + test['Acceleration y']**2 +\
                                       test['Acceleration z']**2)
test['Magnetometer'] =  np.sqrt(test['Magnetometer x']**2 + test['Magnetometer y']**2 +\
                                       test['Magnetometer z']**2)
test['Gyrometer']    =  np.sqrt(test['Gyrometer x']**2 + test['Gyrometer y']**2 +\
                                       test['Gyrometer z']**2)
test['Microphone']    =  (test['Microphone Left Channel Level'] +\
                                test['Microphone Right Channel Level'])/2

test_labels = test['classification']

test = test[['Acceleration','Magnetometer'
                           #'Gyrometer','Microphone','Altimeter (Barometer) Pressure', 
                           ]]

In [135]:
# set window
window = 10
# Rolling Means
glen_H_data['RollingMeanAcceleration'] = glen_H_data['Acceleration'].rolling(window=window,center=False).mean()

# Rolling max
glen_H_data['RollingMaxAcceleration'] = glen_H_data['Acceleration'].rolling(window=window,center=False).max()

# First Derivative
glen_H_data['FirstDdtAcceleration'] = glen_H_data['Acceleration'].diff(1)
glen_H_data['RMFirstDdtAcceleration'] = glen_H_data['Acceleration'].diff(1).rolling(window=window,center=False).mean()

# Rolling var
glen_H_data['RollingVarAcceleration'] = glen_H_data['Acceleration'].rolling(window=window,center=False).var()


In [136]:
# set window
window = 10
# Rolling Means
test['RollingMeanAcceleration'] = test['Acceleration'].rolling(window=window,center=False).mean()

# Rolling Max
test['RollingMaxAcceleration'] = test['Acceleration'].rolling(window=window,center=False).max()

# First Derivative
test['FirstDdtAcceleration'] = test['Acceleration'].diff(1)
test['RMFirstDdtAcceleration'] = test['Acceleration'].diff(1).rolling(window=window,center=False).mean()

# Rolling var
test['RollingVarAcceleration'] = test['Acceleration'].rolling(window=window,center=False).var()


In [137]:
print glen_H_data.shape, test.shape

(6192, 7) (831, 7)


In [150]:
natest = test.isnull().any(axis=1)
#test_re = test

test_X = test[]
test_Y = test_labels
natest

AttributeError: 'DataFrame' object has no attribute 'none'

In [147]:
glen_H_data = glen_H_data.dropna()
glen_H_data

,Acceleration,Magnetometer,RollingMeanAcceleration,RollingMaxAcceleration,FirstDdtAcceleration,RMFirstDdtAcceleration,RollingVarAcceleration
19,0.292521,459.806837,0.299468,1.148418,0.088371,1.116591e-02,0.093864
20,0.053543,459.806837,0.285422,1.148418,-0.238978,-1.404609e-02,0.099128
21,0.040288,459.806837,0.280288,1.148418,-0.013255,-5.134280e-03,0.101603
22,0.099154,459.806837,0.274996,1.148418,0.058865,-5.291546e-03,0.103391
23,0.008780,459.806837,0.244334,1.148418,-0.090374,-3.066247e-02,0.110039
24,0.003380,459.806837,0.223839,1.148418,-0.005401,-2.049450e-02,0.115880
25,0.029039,459.806837,0.200970,1.148418,0.025659,-2.286923e-02,0.119387
26,0.008349,440.277831,0.086963,0.292521,-0.020690,-1.140069e-01,0.009328
27,0.015246,417.673351,0.075445,0.292521,0.006897,-1.151828e-02,0.009542
28,0.007470,417.673351,0.055777,0.292521,-0.007775,-1.966795e-02,0.007785


In [145]:
# load dataset
#dataset = glen_H_data_re.values
glen_H_data = glen_H_data.dropna()
X = glen_H_data
Y = glen_H_data_labels

In [143]:
RF_Class = RandomForestClassifier(n_estimators=500, n_jobs=1)

In [27]:
#x_train,x_test, y_train, y_test = train_test_split(X, encoded_Y, test_size=0.33, random_state=1)

In [146]:
RF_Class.fit(X, Y)
RF_Class.score(X, Y)

ValueError: Number of labels=6192 does not match number of samples=6182

In [89]:
test_guess = RF_Class.predict(test_X)

In [90]:
RF_Class.score(test_X, test_Y)

0.18879415347137637

In [91]:
RF_Class.feature_importances_

array([ 0.01613125,  0.08586514,  0.04477563,  0.0394589 ,  0.16834507,
        0.05387381,  0.03662614,  0.17408715,  0.05604152,  0.05123947,
        0.17045741,  0.0057495 ,  0.00963087,  0.04886399,  0.03885417])